In [ ]:
import cv2
import numpy as np

# File paths for model files
CONFIG_FILE = r'Downloads/ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'
FROZEN_FILE = r'Downloads/frozen_inference_graph.pb'
CLASS_FILE = r'C:\Users\srava\Downloads\coco.names'
VIDEO_FILE = 'Downloads/Group Therapy for Autism Spectrum Disorder.mp4'
TRACKER_TYPE = 'MEDIANFLOW'
CONFIDENCE_THRESHOLD = 0.75

# Load COCO class labels
def load_class_labels(class_file):
    with open(class_file, 'rt') as fp:
        class_labels = fp.read().rstrip('\n').split('\n')
    return class_labels

# Load SSD MobileNet model
def load_model(frozen_file, config_file):
    model = cv2.dnn_DetectionModel(frozen_file, config_file)
    model.setInputSize(340, 340)
    model.setInputScale(1.0 / 127.5)
    model.setInputMean((127.5, 127.5, 127.5))
    model.setInputSwapRB(True)
    return model

# Create a tracker based on the tracker type
def create_tracker_by_name(tracker_type):
    tracker_types = ['BOOSTING', 'MIL', 'KCF', 'TLD', 'MEDIANFLOW', 'MOSSE', 'CSRT']
    if tracker_type == tracker_types[0]:
        return cv2.legacy.TrackerBoosting_create()
    elif tracker_type == tracker_types[1]:
        return cv2.legacy.TrackerMIL_create()
    elif tracker_type == tracker_types[2]:
        return cv2.legacy.TrackerKCF_create()
    elif tracker_type == tracker_types[3]:
        return cv2.legacy.TrackerTLD_create()
    elif tracker_type == tracker_types[4]:
        return cv2.legacy.TrackerMedianFlow_create()
    elif tracker_type == tracker_types[5]:
        return cv2.legacy.TrackerMOSSE_create()
    elif tracker_type == tracker_types[6]:
        return cv2.legacy.TrackerCSRT_create()
    else:
        print('Invalid tracker name!')
        return None

# Initialize trackers with bounding boxes
def track_objects(tracker_type, frame, bboxes):
    trackers = []
    for bbox in bboxes:
        tracker = create_tracker_by_name(tracker_type)
        if tracker is not None:
            tracker.init(frame, bbox)
            trackers.append(tracker)
    return trackers

# Update trackers with the current frame
def update_trackers(trackers, frame):
    updated_bboxes = []
    for tracker in trackers:
        ok, bbox = tracker.update(frame)
        if ok:
            updated_bboxes.append(bbox)
    return updated_bboxes

# Process each frame: detect objects and update trackers
def process_frame(frame, model, class_labels, trackers):
    class_ids, confs, bboxes = model.detect(frame, confThreshold=CONFIDENCE_THRESHOLD)

    # Initialize trackers if objects are detected
    if class_ids is not None and len(bboxes) > 0:
        if trackers is None:
            trackers = track_objects(TRACKER_TYPE, frame, bboxes)
        else:
            updated_bboxes = update_trackers(trackers, frame)
            if updated_bboxes:
                for i, bbox in enumerate(updated_bboxes):
                    (x, y, w, h) = [int(v) for v in bbox]
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
                    cv2.putText(frame, f'ID: {i}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 255, 255), 2)

    return frame, trackers

# Main loop to process the video
def process_video(video_file, model, class_labels):
    cap = cv2.VideoCapture(video_file)

    # Check if video opened successfully
    if not cap.isOpened():
        print("Error: Unable to open video file.")
        return

    trackers = None

    # Loop through video frames
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame, trackers = process_frame(frame, model, class_labels, trackers)

        # Display the frame with detections and tracking
        cv2.imshow('Detection and Tracking', frame)

        # Exit if ESC key is pressed
        if cv2.waitKey(1) & 0xFF == 27:
            break

    cap.release()
    cv2.destroyAllWindows()

# Main function to initialize everything and call other functions
def main():
    # Load the model and class labels
    class_labels = load_class_labels(CLASS_FILE)
    model = load_model(FROZEN_FILE, CONFIG_FILE)

    # Start processing the video
    process_video(VIDEO_FILE, model, class_labels)

# Call the main function
if __name__ == "__main__":
    main()

